# Testing Regression vs. Galerkin Projection

This script tests the accuracy of regression at different PC orders, dimensions, and number of training points.

In [1]:
# Imports
import numpy as np
#import math
from __future__ import print_function
from scipy.stats import qmc

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
#from PyUQTk.utils.func import *

#import sys
#sys.path.append('../pyuqtkarray/')
#import PyUQTk.uqtkarray as uqtkarray

#import polynomial

import pandas as pd

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 4            # Order of the PCE
pc_type = "LU"      # Polynomial type
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
param = nord+1      # Number of quadrature points per dimension for full quadrature or level for sparse quadrature
ndim = 5            # Number of dimensions

We use a pc set object to hold the polynomial we will use. We define our coefficients as a sequence from 1 to 10.

In [3]:
poly = uqtkpce.PCSet("NISPnoq", nord, ndim,pc_type, pc_alpha,pc_beta)
npce=poly.GetNumberPCTerms()
coef=np.array([i+1 for i in range(npce)])
coef.shape

(126,)

We use random testing points in [-1,1].

In [4]:
nTest=npce
rng = qmc.LatinHypercube(d=ndim, seed=42)
rand=2*rng.random(n=nTest)-1

In [5]:
rng2 = np.random.default_rng()
coef=(rng2.random(npce)*10)
coef.shape

(126,)

In order to test regression, we are creating a matrix
Ax=b
A is the evaluated basis
where x are the coefficients,
b are the evaluations of the polynomial.

We create b, which is an array of evaluations of polynomial chaos expansion (a linear combination of the chosen polynomial types, using the chosen coefficients).

In [6]:
# Creating b
b = pce_tools.UQTkEvaluatePCE(poly, coef, rand)

We use regression to create an array of basis evaluations A and find x, our coefficients. If done correctly, x should be the array we originally chose.

In [7]:
# Regression creates A and finds x
c_k=pce_tools.UQTkRegression(poly, b, rand)

Was the regression test successful? Select a decimal place to compare answers to

In [8]:
dec_place=5
reg=(np.round(c_k, dec_place)==np.round(coef, dec_place))
(all(reg) and reg[0]==True)

True

Galerkin projection should produce the original coefficients as well.

In [9]:
#Galerkin for comparison
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim,pc_type, pc_alpha,pc_beta)
#coef=np.array([i+1 for i in range(npce)])

pc_model.SetQuadRule(pc_type, 'full', param)
qdpts, totquat= pce_tools.UQTkGetQuadPoints(pc_model)
f_evals=pce_tools.UQTkEvaluatePCE(pc_model, coef, qdpts)

c_k2 = pce_tools.UQTkGalerkinProjection(pc_model,f_evals)

In [10]:
pd.DataFrame(data=np.transpose([c_k, c_k2, coef]), columns=['Regression', "Galerkin", "coef"])

,Regression,Galerkin,coef
0,1.390146,1.390146,1.390146
1,3.356956,3.356956,3.356956
2,3.466469,3.466469,3.466469
3,4.433141,4.433141,4.433141
4,5.994006,5.994006,5.994006
...,...,...,...
121,2.858060,2.858060,2.858060
122,6.964865,6.964865,6.964865
123,0.839024,0.839024,0.839024
124,8.742857,8.742857,8.742857
